In [1]:
import numpy as np

In [2]:
# 2 feature CAMF. Easily extended to n features.
class collaborative_affinity_metric_fusion():

    def __init__(self):
        pass

    # Creates global affinity matrix by computing the euclidean distance between image feature vectors and applying non-linearity
    # Equation 3
    def affinity_matrix(self, features):
        W = np.array([])
        control_constant = self.get_median(features)


        from sklearn.metrics.pairwise import euclidean_distances
        for feature_a in features:
            for feature_b in features:

                euclidean_dist = euclidean_distances(np.atleast_2d(feature_a),np.atleast_2d(feature_b))
                W = np.append(W,np.exp(euclidean_dist / control_constant))

        return np.reshape(W,(len(features),len(features)))



    # Calculates the median of the distances between feature vectors. Used as a control constant.
    def get_median(self, features):
        distances = np.array([])
        from sklearn.metrics.pairwise import euclidean_distances
        for i in features:
            for j in features:
                distances = np.append(distances,euclidean_distances(np.atleast_2d(i),np.atleast_2d(j)))

        return np.median(distances)




    # Creates a local affinity matrix. If the neighbouring nodes are a minimal neighbour then keep the value. Else = 0.
    # Equation 5.
    def local_affinity_matrix(self, W, neighbours):
        # Iterate over rows
        for row in range(len(W)):
            # Find closest N neighbours
            min_list = np.argsort(W[row])[:neighbours]
            # Find set difference and set larger neighbours to 0
            invalid_neighbours = list(set(np.arange(len(W[row])))-(set(min_list)))
            W[row][invalid_neighbours] = 0
        return W

    

    # Normalise W along each row. 
    def normalise(self, data):
        for row in range(len(data)):
            data[row] = data[row] / np.sum(data[row])
        return np.reshape(data,(1,data.shape[0],data.shape[1]))
    
    

    # Normalise W to give us the status matrix. Global graph. Equation 4.
    def status_matrix_creation(self, W):
        return self.normalise(W)


    # Normalise W_ to give us the status matrix. Local graph. Equation 6.
    def kernel_matrix_creation(self, W_):
        return self.normalise(W_)



    # Creates the status and kernel matrices based on the extracted features and the number of local neighbours
    def create_graphs(self, features, neighbours):
        status_matrix = np.zeros((1,len(features),len(features)))
        kernel_matrix = np.zeros((1,len(features),len(features)))


        W = self.affinity_matrix(features)
        s_matrix = self.status_matrix_creation(W)
        status_matrix = np.concatenate((status_matrix,s_matrix),axis=0)

        from copy import deepcopy
        W_ = self.local_affinity_matrix(deepcopy(W),neighbours)
        k_matrix = self.kernel_matrix_creation(W_)
        kernel_matrix = np.concatenate((kernel_matrix,k_matrix),axis=0)

        return np.delete(status_matrix,0,axis=0), np.delete(kernel_matrix,0,axis=0)
    
    # Find the diffusion indexs due to the fact k!=m. Randomly shuffle until k!=m.
    def status_diffusion_index(self, graphs):
        M = np.arange(graphs)
        np.random.shuffle(M)

        while True:
            if (M != np.arange(graphs)).all():
                break
            else:
                np.random.shuffle(M)
        return M
    

    # Applys cross diffusion to the status matrices and kernel matrices. This merges the features gathered from multiple 
    # algorithms into one matrix that can be queried for image similarity. Equation 7.
    def cross_diffusion(self, status_matrix, kernel_matrix):
        T = 20
        eta = 1
        k = self.status_diffusion_index(status_matrix.shape[0])
        M = status_matrix.shape[0]
        
        # Iterate over feature graphs
        for graph in range(len(k)):
            for t in range(T):
                # Application of equation 7.
                status_matrix[graph] = kernel_matrix[graph] * ((1/M-1) * np.sum(status_matrix[k[graph]])) * kernel_matrix[graph].T + eta * np.identity(kernel_matrix.shape[1])

        return np.mean(status_matrix,axis=0)


    # Merges graphs into ones array to be used in the cross diffusion step.
    def merge_graph(self, graph_1, graph_2, graph_3, graph_4):
        graph = np.concatenate((graph_1, graph_2), axis=0)
        graph = np.concatenate((graph, graph_3), axis=0)
        return np.concatenate((graph, graph_4), axis=0)

    
    # Find the n closest images using W_FAM matrix.
    def image_similarity(self, W_FAM, neighbours):
        image_locations = np.array([])
        
        for row in W_FAM:
            # Locate min n neighbours ignoring the 1st position due to self similarity.
            min_distance = np.argsort(row)[::-1][1 :neighbours+1]
            image_locations = np.append(image_locations, min_distance)
        
        return np.reshape(image_locations,(W_FAM.shape[0],neighbours))

In [9]:
import time
samples = 1000
feature_length = 128
features_algo_1 = np.random.rand(samples,feature_length)
features_algo_2 = np.random.rand(samples,feature_length)
features_algo_3 = np.random.rand(samples,feature_length)
features_algo_4 = np.random.rand(samples,feature_length)

start = time.time()
CAMF = collaborative_affinity_metric_fusion()

status_matrix_1, kernel_matrix_1 = CAMF.create_graphs(features_algo_1, neighbours = 20)
status_matrix_2, kernel_matrix_2 = CAMF.create_graphs(features_algo_2, neighbours = 20)
status_matrix_3, kernel_matrix_3 = CAMF.create_graphs(features_algo_3, neighbours = 20)
status_matrix_4, kernel_matrix_4 = CAMF.create_graphs(features_algo_4, neighbours = 20)


status_matrix = CAMF.merge_graph(status_matrix_1, status_matrix_2, status_matrix_3, status_matrix_4)
kernel_matrix = CAMF.merge_graph(kernel_matrix_1, kernel_matrix_2, kernel_matrix_3, kernel_matrix_4)
W_FAM = CAMF.cross_diffusion(status_matrix, kernel_matrix)
similarity_matrix = CAMF.image_similarity(W_FAM,10)

end = time.time()

print(end-start)

KeyboardInterrupt: 

In [4]:
similarity_matrix = CAMF.image_similarity(W_FAM,10)
print(similarity_matrix)

[[37. 34. 43. 33. 32. 20. 29. 50.  8. 42.]
 [27.  2. 26. 19.  6. 43. 13. 23. 12. 16.]
 [24. 30.  1.  6. 14. 16. 33. 50. 43. 23.]
 [12.  6. 27.  9. 31. 40. 41.  7. 24. 16.]
 [21. 32. 50. 46. 33. 23. 45. 25.  6. 44.]
 [33. 23. 45. 10. 34. 50. 18. 46.  9. 14.]
 [ 2.  3. 24. 43.  1. 38. 28.  4. 10. 46.]
 [19.  9. 12. 33. 16. 11.  6. 47.  3. 32.]
 [49. 50. 39. 24. 45. 17. 47. 46. 37.  0.]
 [35.  3.  7. 31. 24. 10. 25. 19.  5. 15.]
 [50. 18. 12. 43. 13. 31.  5. 40. 34. 35.]
 [29. 47. 24. 40. 22. 13. 25. 12. 10. 18.]
 [ 3. 10. 48. 17. 11. 40.  7.  1. 50.  4.]
 [44. 48. 34. 10. 11. 50.  1. 38.  5. 42.]
 [ 2. 27. 47. 37. 26. 41.  1. 10. 43.  5.]
 [44. 39. 31. 19. 46. 34.  9. 50. 27. 13.]
 [29.  2. 49. 34.  7. 18. 17. 10. 36.  1.]
 [29. 20. 12.  8. 48. 26. 16. 22. 11. 10.]
 [10. 28. 49. 11.  5. 16. 47. 31. 44. 12.]
 [46. 34. 30.  7. 48. 29.  1. 36. 49. 15.]
 [25. 42. 17. 49.  0. 45. 35.  4. 21.  7.]
 [ 4. 25. 45. 29. 44. 10. 20. 28. 26. 22.]
 [11. 27. 39. 24. 31. 17. 50. 33.  4.  9.]
 [ 5. 35. 2